In [ ]:
import os
import json
import time
from datetime import datetime

In [ ]:
def get_data_into_raw(endpoint):
    """
    Fetches data from the specified endpoint and saves it in it's original format in raw layer.
    It also adds two metadata columns (i.e. extract_time & source) to the dataset. Every api call 
    will create a new file in the raw layer. The name of the file has a timestamp which we use to 
    determine which one is the latest dataset to be processed.
    

    Parameters:
    - endpoint (str): The URL endpoint to fetch data from.

    Returns:
    - None
    """
    current_time = datetime.now()

    data = get_data(endpoint)
    data['extract_time'] = datetime.now().timestamp()
    data['source'] = 'exchange rates api'

    with open(f'../../data/raw/raw_rates_{current_time}.json', 'w') as json_file:
        json.dump(data, json_file)

In [ ]:
def get_data_into_structured():
    """
    Reads the latest raw JSON data file from the raw layer, transform it, enforce schema and write 
    it into structured layer. Here we also add a number of metadata columns (i.e. source_file, 
    extract_time & current) to the dataset. In this layer we perform a crude SCD with a column 
    named 'current' which holds a '1' for latest records and a '0' for historical records.

    Parameters:
    - None
    
    Returns:
    - None
    """
    # First we need to find the latest file in the raw zone
    directory = '../../data/raw/'
    files = os.listdir(directory)
    
    if not files:
        return None
        
    latest_file = max(files, key=lambda f: os.path.getmtime(os.path.join(directory, f)))

    # We then extract the rates from this file, clean and enforce schema and load it into a dataframe
    with open(f'{directory}/{latest_file}', "r") as file:
        json_data = json.load(file)
    
    new_rates_df = get_rates_dataframe(json_data)

    extract_time = datetime.now().timestamp()
    new_rates_df = new_rates_df.assign(source_file=lambda x: latest_file)
    new_rates_df = new_rates_df.assign(extract_time=lambda x: extract_time)

    # current = 1 for records in the latest batch
    new_rates_df = new_rates_df.assign(current=lambda x: 1) 

    # If structured has previouosly been processed we set current = 0 for all records
    structured_path = '../../data/structured/structured_rates.json'
    if os.path.exists(structured_path):
        with open(structured_path, 'r') as file:
            data = json.load(file)

        old_rates_df = pd.DataFrame(data)
        old_rates_df['current'] = 0
        
        combined_df = pd.concat([old_rates_df, new_rates_df], ignore_index=True)

        os.remove(structured_path)
    else:
        combined_df = new_rates_df

    combined_df.to_json(structured_path, orient='records')

In [ ]:
def get_data_into_curated():
    """
    Reads the structured file from structured layer. We perform aggregation here.

    Parameters:
    - None
    
    Returns:
    - None
    """
    # Read latest rates from structured_rates.json
    structured_path = '../../data/structured/structured_rates.json'
    if not os.path.exists(structured_path):
        return
        
    with open(structured_path, 'r') as file:
        data = json.load(file)

    structured_rates_df = pd.DataFrame(data)

    latest_df = structured_rates_df[structured_rates_df['current'] == 1]

    summary_path = '../../data/curated/summary_rates.json'
    if os.path.exists(summary_path):
        os.remove(summary_path)

    statistics_df = latest_df.groupby(['currency'], as_index=False).agg(
        mean_rate=('rate', 'mean'),
        worst_rate=('rate', 'min'),
        best_rate=('rate', 'max')
    )

    statistics_df.to_json(summary_path, orient='records')

In [ ]:
def run_pipeline(endpoint):
    """
    Orchestrates the end to end pipeline, brings data from source api into raw, structured & curated layers.

    Parameters:
    - endpoint (str): The URL endpoint to fetch data from.
    
    Returns:
    - None
    """
    get_data_into_raw(endpoint)
    get_data_into_structured()
    get_data_into_curated()